# Notebook exploring tool usage

In [14]:
import os
from dotenv import load_dotenv

while 'notebooks' in os.getcwd():
    os.chdir('..')

load_dotenv('.env')

True

In [155]:
import json
import random
from typing import Union

from IPython.display import display, Markdown

from openai import OpenAI

## Raw LLM

In [63]:
class LLM:
    def __init__(self, model: str = 'gpt-4.1'):
        self.client = OpenAI()
        self.model = model
    
        
    def __call__(self, messages: Union[str, list[dict[str, str]]]):
        if isinstance(messages, str):
            messages = [{'role': 'user', 'content': messages}]
        result = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
        )
        return result.choices[0].message.content

In [64]:
llm = LLM()

In [65]:
llm('What up')

"Hey! Not much, just here to help. What's up with you?"

## Web search (built-in openai tool)

In [122]:
%%time
response = llm.client.responses.create(
    model=llm.model,
    tools=[{"type": "web_search_preview"}],
    input="When is the next UFC?"
)

CPU times: user 11.4 ms, sys: 3.27 ms, total: 14.6 ms
Wall time: 5.07 s


In [123]:
response.usage.model_dump()

{'input_tokens': 307,
 'input_tokens_details': {'cached_tokens': 0},
 'output_tokens': 335,
 'output_tokens_details': {'reasoning_tokens': 0},
 'total_tokens': 642}

In [128]:
print(json.dumps(response.model_dump(), indent=3, ensure_ascii=False))

{
   "id": "resp_6883c544de2081a093db338a88be6e7e0d3ec13ef050dafb",
   "created_at": 1753466180.0,
   "error": null,
   "incomplete_details": null,
   "instructions": null,
   "metadata": {},
   "model": "gpt-4.1-2025-04-14",
   "object": "response",
   "output": [
      {
         "id": "ws_6883c54534d881a0b482deaa1bc686080d3ec13ef050dafb",
         "action": {
            "query": "next UFC event date",
            "type": "search"
         },
         "status": "completed",
         "type": "web_search_call"
      },
      {
         "id": "msg_6883c546f11c81a08396d6e56782a0130d3ec13ef050dafb",
         "content": [
            {
               "annotations": [
                  {
                     "end_index": 289,
                     "start_index": 172,
                     "title": "UFC Schedule: Upcoming cards, main events, host cities",
                     "type": "url_citation",
                     "url": "https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-eve

In [132]:
response.output[0]

ResponseFunctionWebSearch(id='ws_6883c54534d881a0b482deaa1bc686080d3ec13ef050dafb', action=ActionSearch(query='next UFC event date', type='search'), status='completed', type='web_search_call')

In [133]:
response.output[1]

ResponseOutputMessage(id='msg_6883c546f11c81a08396d6e56782a0130d3ec13ef050dafb', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=289, start_index=172, title='UFC Schedule: Upcoming cards, main events, host cities', type='url_citation', url='https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai'), AnnotationURLCitation(end_index=572, start_index=455, title='UFC Schedule: Upcoming cards, main events, host cities', type='url_citation', url='https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai'), AnnotationURLCitation(end_index=1040, start_index=872, title='Aspinall to defend UFC heavyweight title against Gane', type='url_citation', url='https://www.reuters.com/sports/aspinall-defend-ufc-heavyweight-title-against-gane-2025-07-23/?utm_source=openai'), AnnotationURLCitation(end_index=1197, start_index=1043, title='Adesanya se moja sobre UFC 319: "Será caótico"', type='url_cit

In [135]:
display(Markdown(response.output[1].content[0].text))

The next UFC event is scheduled for Saturday, July 26, 2025, featuring a middleweight bout between Robert Whittaker and Reinier de Ridder at the Etihad Arena in Abu Dhabi. ([sportsnet.ca](https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai))

Following that, UFC 319 is set for August 16, 2025, at the United Center in Chicago, with a middleweight title fight between Dricus du Plessis and Khamzat Chimaev. ([sportsnet.ca](https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai))

For viewers in the United States, UFC events are typically broadcast on ESPN and ESPN+, with pay-per-view options available for major events. Please check your local listings or the official UFC website for the most up-to-date broadcast information.


## Upcoming UFC Events and Fighter Updates:
- [Aspinall to defend UFC heavyweight title against Gane](https://www.reuters.com/sports/aspinall-defend-ufc-heavyweight-title-against-gane-2025-07-23/?utm_source=openai)
- [Adesanya se moja sobre UFC 319: "Será caótico"](https://as.com/masdeporte/polideportivo/adesanya-se-moja-sobre-ufc-319-sera-caotico-n/?utm_source=openai) 

In [142]:
response.output[1].content[0].annotations

[AnnotationURLCitation(end_index=289, start_index=172, title='UFC Schedule: Upcoming cards, main events, host cities', type='url_citation', url='https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai'),
 AnnotationURLCitation(end_index=572, start_index=455, title='UFC Schedule: Upcoming cards, main events, host cities', type='url_citation', url='https://www.sportsnet.ca/ufc/ufc-schedule-upcoming-cards-main-events-host-cities/?utm_source=openai'),
 AnnotationURLCitation(end_index=1040, start_index=872, title='Aspinall to defend UFC heavyweight title against Gane', type='url_citation', url='https://www.reuters.com/sports/aspinall-defend-ufc-heavyweight-title-against-gane-2025-07-23/?utm_source=openai'),
 AnnotationURLCitation(end_index=1197, start_index=1043, title='Adesanya se moja sobre UFC 319: "Será caótico"', type='url_citation', url='https://as.com/masdeporte/polideportivo/adesanya-se-moja-sobre-ufc-319-sera-caotico-n/?utm_source=openai'

In [146]:
tool = response.tools[0]
type(tool)

openai.types.responses.web_search_tool.WebSearchTool

## Function calling tool
Probably the most useful tool of letting LLM to decide to use your own function.

In [148]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

In [149]:
%%time
response = llm.client.responses.create(
    model=llm.model,
    tools=tools,
    input="What is the weather in Warsaw, Poland"
)

CPU times: user 14.5 ms, sys: 3.46 ms, total: 17.9 ms
Wall time: 823 ms


In [152]:
len(response.output)

1

In [158]:
function_call_response = response.output[0]
function_call_response

ResponseFunctionToolCall(arguments='{"location":"Warsaw, Poland"}', call_id='call_r7WXRGTjLwlVZIqz8I1cowag', name='get_weather', type='function_call', id='fc_6883ca544170819facff23dac0c8d87205829bb588582655', status='completed')

In [159]:
def get_weather(location: str):
    return f"Weather in {location} is {random.randint(0, 40)} °C"

In [160]:
functions = {
    'get_weather': get_weather
}

In [163]:
function_call_response.arguments

'{"location":"Warsaw, Poland"}'

In [166]:
functions[function_call_response.name](**json.loads(function_call_response.arguments))

'Weather in Warsaw, Poland is 23 °C'